In [2]:
import pandas as pd
import numpy as np

In [20]:
# Read large datasets: https://www.kaggle.com/rohanrao/tutorial-on-reading-large-datasets
train_dtypes = {
    "row_id": "int64",
    "timestamp": "int64",
    "user_id": "int32",
    "content_id": "int16",
    "content_type_id": "boolean",
    "task_container_id": "int16",
    "user_answer": "int8",
    "answered_correctly": "int8",
    "prior_question_elapsed_time": "float32",
    "prior_question_had_explanation": "boolean"
}

train_df = pd.read_csv('train.csv', nrows=100000, dtype=train_dtypes)
questions_df = pd.read_csv('questions.csv')
lectures_df = pd.read_csv('lectures.csv')
example_test_df = pd.read_csv('example_test.csv')

In [4]:
train_df

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,0,115,5692,False,1,3,1,NaN,<NA>
1,1,56943,115,5716,False,2,2,1,37000.0,False
2,2,118363,115,128,False,0,0,1,55000.0,False
3,3,131167,115,7860,False,3,0,1,19000.0,False
4,4,137965,115,7922,False,4,1,1,11000.0,False
...,...,...,...,...,...,...,...,...,...,...
99995,99995,153647401,2078569,4334,False,275,3,0,6000.0,True
99996,99996,153692472,2078569,6436,False,276,3,0,9000.0,True
99997,99997,153722998,2078569,6446,False,277,2,1,21000.0,True
99998,99998,153759775,2078569,3715,False,278,3,0,12000.0,True


In [5]:
questions_df

,question_id,bundle_id,correct_answer,part,tags
0,0,0,0,1,51 131 162 38
1,1,1,1,1,131 36 81
2,2,2,0,1,131 101 162 92
3,3,3,0,1,131 149 162 29
4,4,4,3,1,131 5 162 38
...,...,...,...,...,...
13518,13518,13518,3,5,14
13519,13519,13519,3,5,8
13520,13520,13520,2,5,73
13521,13521,13521,0,5,125


In [6]:
lectures_df

,lecture_id,tag,part,type_of
0,89,159,5,concept
1,100,70,1,concept
2,185,45,6,concept
3,192,79,5,solving question
4,317,156,5,solving question
...,...,...,...,...
413,32535,8,5,solving question
414,32570,113,3,solving question
415,32604,24,6,concept
416,32625,142,2,concept


In [7]:
example_test_df

,row_id,group_num,timestamp,user_id,content_id,content_type_id,task_container_id,prior_question_elapsed_time,prior_question_had_explanation,prior_group_answers_correct,prior_group_responses
0,0,0,0,275030867,5729,0,0,NaN,NaN,[],[]
1,1,0,13309898705,554169193,12010,0,4427,19000.0,True,NaN,NaN
2,2,0,4213672059,1720860329,457,0,240,17000.0,True,NaN,NaN
3,3,0,62798072960,288641214,13262,0,266,23000.0,True,NaN,NaN
4,4,0,10585422061,1728340777,6119,0,162,72400.0,True,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
99,104,3,13167339284,1900527744,3004,0,1179,24667.0,True,NaN,NaN
100,105,3,13167339284,1900527744,3003,0,1179,24667.0,True,NaN,NaN
101,106,3,64497673060,7792299,7908,0,676,19000.0,True,NaN,NaN
102,107,3,62798166743,288641214,9077,0,269,25000.0,True,NaN,NaN


In [22]:
questions_df['tags'] = questions_df['tags'].apply(lambda s: [int(x) for x in str(s).split() if x != 'nan'])

In [29]:
NUM_TAGS = len(lectures_df['tag'].unique())

In [31]:
NUM_TAGS

151

In [32]:
pd.get_dummies(lectures_df['tag'])

,0,1,2,4,5,6,7,8,9,12,...,176,177,178,179,180,181,182,183,184,187
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
414,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
415,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
416,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
user_tags = train_df[train_df['content_type_id'] == 1].groupby('user_id').apply(lambda x: list(x['content_id']))

user_id
2746                                                  [6808]
5382                                   [16736, 30207, 18545]
8623                                   [10540, 25843, 29544]
12741              [20307, 10688, 10217, 3852, 18186, 22792]
13134         [21887, 29544, 3153, 28921, 28349, 2278, 7477]
                                 ...                        
2035385                                 [3153, 21887, 14443]
2041025    [27845, 2278, 22034, 32018, 27339, 6481, 23165...
2048552    [10540, 32736, 21168, 25843, 18467, 30276, 851...
2066347                                               [7926]
2073472                                              [29695]
Length: 144, dtype: object

In [ ]:
'''
Build more robust embeddings by learning how certain tags help answer certain questions.
So gather tags student has learned in lectures -> learn which tags helped answer the question the most.
'''